<a href="https://colab.research.google.com/github/AlexKazmin/SkillFactory-Projects/blob/main/Project4_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D0%B0_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Подготовка основы рекомендательной системы**
# ЦЕЛЬ
Подготовить основу рекомендательной системы для онлайн-школы, благодаря которой можно будет предлагать клиентам интересные им курсы.


# Задачи:
*  Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL
*  Обработать данные средствами Python
*  Составить итоговую таблицу с рекомендациями

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 
import itertools
from collections import Counter

# Анализ и подготовка данных

Посмотрим, за какой период в наших данных есть продажи.

In [ ]:
def getQuerySQL1():
    query = """
    SELECT 
    min(date(updated_at)),
    max(date(updated_at))
    FROM final.carts 
    WHERE state = 'successful'
    """.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

date_salse = pd.DataFrame(getQuerySQL1())
date_salse

,min,max
0,2017-01-01,2018-12-30


Посмотрим, сколько клиентов покупали курсы.

In [ ]:

def getClientPurchasedCourses():
    query = '''
    SELECT COUNT (DISTINCT user_id) 
    FROM final.carts c
    LEFT JOIN final.cart_items ci ON c.id = ci.cart_id
    WHERE ci.resource_type = 'Course'
      AND c.state = 'successful'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
client_purchased_courses = getClientPurchasedCourses()
print('За данный период ', client_purchased_courses[0]['count'], 'клиента купили курсы')

За данный период  49006 клиента купили курсы


Сколько всего есть различных курсов?

In [ ]:

def getVariousCourses():
    query = '''
    SELECT 
        count (distinct ci.resource_id) 
    FROM final.carts c
    LEFT JOIN final.cart_items ci ON c.id = ci.cart_id
    WHERE ci.resource_type = 'Course' 
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
various_courses = getVariousCourses()
print('Всего в онлайн-школе ', various_courses[0]['count'], 'различных курсов')

Всего в онлайн-школе  127 различных курсов


Вычислим среднее число купленных курсов на одного клиента


In [ ]:
def getAverageAmountCourses():
    query = '''
    WITH data as
    (
    SELECT DISTINCT
       user_id,
       count(user_id) as count_course
    FROM final.carts fc
    LEFT JOIN final.cart_items fci ON fc.id = fci.cart_id
    WHERE fci.resource_type = 'Course' AND fc.state = 'successful'
    GROUP BY 1
    )
    SELECT 
    avg(count_course) 
    FROM data
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
average_amount_courses = getAverageAmountCourses()
print('Среднее количество курсов на одного клиента составляет ', round(average_amount_courses[0]['avg'], 2))

Среднее количество курсов на одного клиента составляет  1.44


Сколько клиентов купили больше одного курса?

In [ ]:
def getClientBuyMoreOneCourses():
    query = '''
    WITH clients AS --CTE для определения пользователей, купивших курсы

      (SELECT DISTINCT fc.user_id,
                   fci.resource_id
   FROM final.carts fc
   JOIN final.cart_items fci ON fc.id=fci.cart_id
   WHERE fc.state = 'successful'
     AND fci.resource_type='Course'
   ORDER BY fc.user_id),
   
   orders AS --CTE для подсчета количества курсов, купленных пользователем

  (SELECT user_id,
          count(resource_id) AS cnt_course
   FROM clients
   GROUP BY 1)
   
    SELECT count(user_id)
    FROM orders
    WHERE cnt_course>1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
client_buy_more_one_courses = getClientBuyMoreOneCourses()
print(client_buy_more_one_courses[0]['count'], ' клиента купили более одного курса')

12656  клиента купили более одного курса


Выгружаем в датафрейм данные по продажам курсов в разрезе пользователей (купивших более одного курса)

In [ ]:
def getDataUpload():
    query = '''
    WITH clients AS --CTE для определения пользователей, купивших курсы
    
    (SELECT DISTINCT fc.user_id,
                   fci.resource_id
       FROM final.carts fc
       JOIN final.cart_items fci ON fc.id=fci.cart_id
       WHERE fc.state = 'successful'
         AND fci.resource_type='Course'
       ORDER BY fc.user_id), 
       
       orders AS --CTE для подсчета количества курсов, купленных пользователем

    (SELECT user_id,
          count(resource_id) AS cnt_course
      FROM clients
      GROUP BY 1)

    SELECT o.user_id,
           fci.resource_id
    FROM orders o
    JOIN final.carts fc ON o.user_id=fc.user_id
    JOIN final.cart_items fci ON fc.id=fci.cart_id
    WHERE cnt_course>1
      AND fc.state = 'successful'
      AND fci.resource_type='Course'
    ORDER BY  o.user_id
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
df = pd.DataFrame(getDataUpload())

df.head()


,user_id,resource_id
0,51,1099
1,51,516
2,6117,356
3,6117,1125
4,6117,357


## Обработка данных

Сохраняем список уникальных курсов

In [ ]:
courses = df['resource_id'].unique() 

По каждому пользователю выводим список курсов

In [ ]:
user_df = df.groupby('user_id')['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
user_df.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


При помощи функции combinations из модуля itertools, создаем список пар курсов, купенные одним пользователем

In [ ]:
list_course = list()
for resource in user_df['resource_id']:
    for pair_course in itertools.combinations(resource,2):
        list_course.append(pair_course)
len(list_course)

40017

С помощью функции Counter создаём список всех уникальных пар и количество их повторений

In [ ]:
count_list_cours = Counter(list_course)
# count_list_cours

На основе списка создаём датафрейм.               


Первый столбец - пара, второй - количество повторений пары

In [ ]:
cours_pair_df = pd.DataFrame.from_dict(count_list_cours, orient='index').reset_index()
cours_pair_df.columns = ['pair','count_purc']

cours_pair_df


,pair,count_purc
0,"(516, 1099)",25
1,"(356, 357)",100
2,"(356, 1125)",44
3,"(357, 1125)",52
4,"(553, 1147)",16
...,...,...
3984,"(741, 791)",1
3985,"(361, 1128)",1
3986,"(743, 1156)",1
3987,"(864, 1129)",1


Анализируем датафрейм

In [ ]:
cours_pair_df.describe()

,count_purc
count,3989.000000
mean,10.031838
std,26.355998
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,797.000000


Среднее значение повторений пар 10. 

75% пар имеет частоту повторений меньше 10, поэтому все, что ниже третьего квартиля можно отброисть. Возьмем только 25% наиболее часто повторяющихся пар






При помощи percentile отбираем самые популярные 25% пар, записываем их в словарь

In [ ]:
percent = np.percentile(cours_pair_df['count_purc'],75)
popular_pair_dict = {k: v for k, v in count_list_cours.items() if v > percent}


Создаём  функцию recommend, которая перебирает ключи словаря popular_pair_dict, подбирая подходящую рекомендацию

In [ ]:
def recommend (course):
    x = []
    for i in popular_pair_dict.keys():
        if i [0] == course:
            x.append((i,popular_pair_dict[i]))
    scl = sorted( x, key=lambda x: x[1], reverse = True)
    return scl[:2]


Для курсов, которым не подобралась популярная пара, создаём рекомендацию из самой популярной пары курсов

In [ ]:
is_not_pair_1 = cours_pair_df.sort_values('count_purc', ascending= False).iloc[0][0][0]
is_not_pair_2 = cours_pair_df.sort_values('count_purc', ascending= False).iloc[0][0][1]

# Составяем итоговую таблицу рекомендаций

Создаем датафрейм и заполняем его функцией recommend





In [ ]:
recomend_df = pd.DataFrame(columns = ['recommendation_1','recommendation_2']) 


for i in courses:
    if len(recommend(i)) == 2:  # если выдает 2 рекомендации для курса
        recomend_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]] # записываем обе рекомендации
    elif len(recommend(i)) == 1: # если выдает 1 рекомендацию для курса
        recomend_df.loc[i] = [recommend(i)[0][0][1], is_not_pair_2] # записываем одну рекомендацию, вторую берем из самой популярной пары
    else:
        recomend_df.loc[i] = [is_not_pair_1, is_not_pair_2] # если ничего не выдает, берем самую популярную пару




Переносим номер курса из индекса в отдельную колонку

In [ ]:
recomend_df=recomend_df.reset_index()
recomend_df=recomend_df.rename(columns = {'index' : 'course'})
pd.set_option('display.max_rows', 130)

recomend_df


,course,recommendation_1,recommendation_2
0,1099,1139,1187
1,516,745,553
2,356,571,357
3,1125,1186,1144
4,357,571,1125
5,553,745,568
6,1147,551,566
7,1138,551,566
8,361,551,1138
9,1140,1185,1141


Скрипт для проверки поиска рекомендаций по номеру введённого курса

In [ ]:
course_search = int(input('Номер курса  '))
if course_search in courses:
    info = recomend_df[recomend_df['course'] == course_search]
    print('Рекомендованые курсы:', info.iloc[0][1],'и',info.iloc[0][2])
else:
    print('Курс не найден')

Номер курса  553
Рекомендованые курсы: 745 и 568


# Вывод

Создана таблица с рекомендациями для 126 курсов. 1 курс не попал еще на этапе отбора данных, так как его ни разу не покупали.
Для тех курсов, которые редко покупаются, рекомендуются самые популярные курсы.



